In [1]:
import sys
# sys.path.append("/home/test/PycharmProjects")
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap
import pandas as pd
import json
import copy
import numpy as np


In [2]:
# pd.set_option('display.max_rows', None)

# d_path = '/raid/AISSEL/htest/datasets/ped_data/ped_ftest/all'
# before
base_path = '.'
df_b = pd.read_pickle(f'{base_path}/pd_base.pk')

# after q50_over_avg
d_path = '/raid/AISSEL/htest/datasets/ped_data/ped_ftest/missed_q50_over_avg'
df_f = pd.read_pickle(f'{d_path}/pd_mq50_over_avg.pk')


# wit
d_path = '/raid/AISSEL/htest/datasets/ped_data/wit/missed_q50_over_avg'
df_w = pd.read_pickle(f'{d_path}/ped_mq50_over_avg.pk')
# df

# random
d_path = '/raid/AISSEL/htest/datasets/ped_data/wit/random_missed'
df_r = pd.read_pickle(f'{d_path}/ped_mq50_over_avg.pk')

In [3]:
annotation_file = './captions_val2014.json'
with open(annotation_file, 'r') as f:
    jdata = json.load(f)

In [4]:
def create_gt(tag):
    context = []
    for el in tag:
        if tag[el]:
            context.append(el)
    if not context:
        context = None
    return context

In [5]:
def id2imdf(_df):
    _df = _df[['uniq_id']]
    _df['license'] = 1
    _df['url'] = 'http://farm9.staticflickr.com'
    _df['file_name'] = 'test.jpg'
    _df['width'] = 640
    _df['height'] = 360
    _df['date_captured'] = '2013-11-14 11:18:45'
    _df.columns = _df.columns.str.replace('uniq_id', 'id')
    return _df

In [6]:
def concat_gt2anno(_df):
    image_id = []
    caption = []
    for index, row in _df.iterrows():
        image_id.append(row['image_id'])
        caption.append(row['gt'])
    
    df_ = pd.DataFrame()
    df_['image_id'] = image_id
    df_['id'] = np.arange(len(caption))
    df_['caption'] = caption
    return df_


In [7]:
def gt2anno(_df):
    image_id = []
    caption = []
    for index, row in _df.iterrows():
#         print(row['image_id'], row['gt'])
        for el in row['tag_list']:
            image_id.append(row['image_id'])
            caption.append(el)
#         break
    df_ = pd.DataFrame()
    df_['image_id'] = image_id
    df_['id'] = np.arange(len(caption))
    df_['caption'] = caption
    return df_


In [8]:
def web_2anno(_df):
    image_id = []
#     caption = []
    for index, row in _df.iterrows():
        image_id.append(row['image_id'])
#         caption.append('pedestrian')
    
    df_ = pd.DataFrame()
    df_['image_id'] = image_id
    df_['id'] = np.arange(len(image_id))
    df_['caption'] = _df['caption'].to_list()
    return df_

In [9]:
def h_gt2anno(_df):
    image_id = []
    caption = []
    for index, row in _df.iterrows():
        image_id.append(row['image_id'])
        caption.append(row['h_caption'])
    
    df_ = pd.DataFrame()
    df_['image_id'] = image_id
    df_['id'] = np.arange(len(caption))
    df_['caption'] = caption
    return df_


In [10]:
def create_dict(df_wp):
#     df_wp = pd.read_pickle(df_wp)
    df_n = df_wp.drop_duplicates(subset='uniq_id', keep="last")
    df_n = df_n.reset_index(drop=True)
    
    gk = df_wp.groupby('topics')
    pg_dict = dict()
    for el in gk.groups.keys():
        pg_dict[el] = dict()
        pg_dict[el]['data'] = gk.get_group(el)
    #     pg_dict[el]['data'] = pg_dict[el]['data'][pg_dict[el]['data']['topic2caption_sim'] >= pg_dict[el]['data']['topic2caption_sim'].quantile(.25)]
        pg_dict[el]['data'] = pg_dict[el]['data'].reset_index(drop=True)
        temp = pg_dict[el]['data'][['image_id', 'ofa_caption']]
        temp.columns = temp.columns.str.replace('ofa_caption', 'caption')
        pg_dict[el]['results'] = temp
        pg_dict[el]['images'] = id2imdf(pg_dict[el]['data'])
    #     pg_dict[el]['split_annotations'] = gt2anno(pg_dict[el]['data'])
        pg_dict[el]['human_annotations'] = h_gt2anno(pg_dict[el]['data'])
        pg_dict[el]['con_annotations'] = concat_gt2anno(pg_dict[el]['data'])
        pg_dict[el]['web_annotations'] = web_2anno(pg_dict[el]['data'])
    return df_n, pg_dict

In [11]:
def crate_json(pg_dict, el, t='human_annotations'):
    t_data = dict()
    t_data['info'] = jdata['info']
    t_data['images'] = pg_dict[el]['images'].to_dict('records')
    t_data['type'] = jdata['type']
    t_data['licenses'] = jdata['licenses']
    t_data['annotations'] = pg_dict[el][t].to_dict('records')
    return t_data
    

In [12]:
def file_path_dict(pg_dict, test='wit'):
    file_dict = dict()
    for el in pg_dict:
        file_dict[el] = dict()
#         print(el)

        s_data = crate_json(pg_dict, el,t='human_annotations')
        c_data = crate_json(pg_dict, el,t='con_annotations')  
        o_data = crate_json(pg_dict, el,t='web_annotations')  
        res = pg_dict[el]['results'].to_dict('records')

        file_dict[el]['results'] = f"data/{test}_{el}_results.json"
        with open(file_dict[el]['results'], "w") as final:
            json.dump(res, final)

        file_dict[el]['sval'] = f"data/{test}_{el}_sval.json"
        with open(file_dict[el]['sval'], "w") as final:
            json.dump(s_data, final)

        file_dict[el]['cval'] = f"data/{test}_{el}_cval.json"
        with open(file_dict[el]['cval'], "w") as final:
            json.dump(c_data, final)

        file_dict[el]['oval'] = f"data/{test}_{el}_oval.json"
        with open(file_dict[el]['oval'], "w") as final:
            json.dump(o_data, final)
    return file_dict

In [13]:
def calc_metrics(file_dict, topic='a', anno='s'):
    if anno == 's':
        annotation_file = file_dict[topic]['sval']
    elif anno == 'c':
        annotation_file = file_dict[topic]['cval']
    else:
        annotation_file = file_dict[topic]['oval']
        
    results_file = file_dict[topic]['results']
    coco = COCO(annotation_file)
    coco_result = coco.loadRes(results_file)
    coco_eval = COCOEvalCap(coco, coco_result)
    coco_eval.params['image_id'] = coco_result.getImgIds()
    coco_eval.evaluate()
    return coco_eval.eval


# Base

In [14]:
df_b1, pg_dict_b = create_dict(df_b)
# gk = df_wp.groupby('topics')
# pg_dict = dict()
# for el in gk.groups.keys():
all_dict = dict()
all_dict['data'] = copy.deepcopy(df_b1)
all_dict['data'] = all_dict['data'].reset_index(drop=True)
temp = all_dict['data'][['image_id', 'ofa_caption']]
temp.columns = temp.columns.str.replace('ofa_caption', 'caption')
all_dict['results'] = temp
all_dict['images'] = id2imdf(all_dict['data'])
all_dict['annotations'] = web_2anno(all_dict['data'])

t_data = dict()
t_data['info'] = jdata['info']
t_data['images'] = all_dict['images'].to_dict('records')
t_data['type'] = jdata['type']
t_data['licenses'] = jdata['licenses']
t_data['annotations'] = all_dict['annotations'].to_dict('records')
file_p = f"data/all_anno.json"
with open(file_p, "w") as final:
    json.dump(t_data, final)
res = all_dict['results'].to_dict('records')
file_p = f"data/all_results.json"
with open(file_p, "w") as final:
    json.dump(res, final)

annotation_file = 'data/all_anno.json'
results_file = 'data/all_results.json'

# create coco object and coco_result object
coco = COCO(annotation_file)
coco_result = coco.loadRes(results_file)

# create coco_eval object by taking coco and coco_result
coco_eval = COCOEvalCap(coco, coco_result)

# evaluate on a subset of images by setting
# coco_eval.params['image_id'] = coco_result.getImgIds()
# please remove this line when evaluating the full validation set
coco_eval.params['image_id'] = coco_result.getImgIds()

# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
coco_eval.evaluate()

# # print output evaluation scores
# for metric, score in coco_eval.eval.items():
#     print(f'{metric}: {score:.3f}')

/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying t

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 5370, 'reflen': 7910, 'guess': [5370, 4813, 4256, 3699], 'correct': [1068, 181, 45, 15]}
ratio: 0.6788874841971329
Bleu_1: 0.124
Bleu_2: 0.054
Bleu_3: 0.027
Bleu_4: 0.015
computing METEOR score...
METEOR: 0.059
computing Rouge score...
ROUGE_L: 0.133
computing CIDEr score...
CIDEr: 0.216
computing SPICE score...
SPICE: 0.088


In [16]:
file_dict_b = file_path_dict(pg_dict_b, test='base')
web_res_dict = dict()
for el in pg_dict_b:
    web_res_dict[el] = calc_metrics(file_dict_b, topic=el, anno='o')

all_res_b = []
for el in pg_dict_b:
    all_res_b.append(web_res_dict[el])
df_res = pd.DataFrame(all_res_b)
display(df_res.describe().round(2))

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 491, 'reflen': 766, 'guess': [491, 440, 389, 338], 'correct': [79, 11, 2, 0]}
ratio: 0.6409921671009908
Bleu_1: 0.092
Bleu_2: 0.036
Bleu_3: 0.016
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.040
computing Rouge score...
ROUGE_L: 0.104
computing CIDEr score...
CIDEr: 0.148
computing SPICE score...
SPICE: 0.075
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 971, 'reflen': 1454, 'guess': [971, 868, 765, 662], 'correct': [198, 29, 5, 3]}
ratio: 0.6678129298482339
Bleu_1: 0.124
Bleu_2: 0.050
Bleu_3: 0.022
Bleu_4: 0.013
computing METEOR score...
METEOR: 0.063
comput

SPICE: 0.095


,Bleu_1,Bleu_2,Bleu_3,Bleu_4,METEOR,ROUGE_L,CIDEr,SPICE
count,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.00
mean,0.11,0.04,0.02,0.01,0.05,0.12,0.18,0.07
std,0.03,0.02,0.01,0.01,0.02,0.04,0.06,0.04
min,0.06,0.00,0.00,0.00,0.01,0.05,0.10,0.00
25%,0.09,0.03,0.01,0.00,0.04,0.10,0.14,0.04
50%,0.10,0.04,0.02,0.00,0.05,0.11,0.17,0.07
75%,0.12,0.05,0.03,0.01,0.07,0.15,0.24,0.09
max,0.16,0.09,0.04,0.02,0.09,0.21,0.28,0.16


In [17]:
all_topics_b = []
for k in web_res_dict:
    d_temp = copy.deepcopy(web_res_dict[k])
    del d_temp['Bleu_4']
    del d_temp['Bleu_2']
    del d_temp['Bleu_3']
    d_temp['topic'] = k
    all_topics_b.append(d_temp)
#     break
# d_temp
df_tmp = pd.DataFrame(all_topics_b)
display(df_tmp.round(2))

,Bleu_1,METEOR,ROUGE_L,CIDEr,SPICE,topic
0,0.09,0.04,0.10,0.15,0.07,a
1,0.12,0.06,0.13,0.24,0.08,b
2,0.14,0.07,0.16,0.27,0.14,c
3,0.12,0.07,0.16,0.17,0.12,d
4,0.09,0.05,0.12,0.10,0.07,e
5,0.10,0.03,0.10,0.15,0.04,f
6,0.10,0.04,0.10,0.14,0.05,g
7,0.12,0.06,0.12,0.11,0.06,h
8,0.15,0.07,0.17,0.26,0.09,i
9,0.06,0.01,0.05,0.13,0.00,j


# Fat

In [18]:
df_f1, pg_dict_f = create_dict(df_f)
all_dict = dict()
all_dict['data'] = copy.deepcopy(df_f1)
all_dict['data'] = all_dict['data'].reset_index(drop=True)
temp = all_dict['data'][['image_id', 'ofa_caption']]
temp.columns = temp.columns.str.replace('ofa_caption', 'caption')
all_dict['results'] = temp
all_dict['images'] = id2imdf(all_dict['data'])
all_dict['annotations'] = web_2anno(all_dict['data'])

t_data = dict()
t_data['info'] = jdata['info']
t_data['images'] = all_dict['images'].to_dict('records')
t_data['type'] = jdata['type']
t_data['licenses'] = jdata['licenses']
t_data['annotations'] = all_dict['annotations'].to_dict('records')
file_p = f"data/all_anno.json"
with open(file_p, "w") as final:
    json.dump(t_data, final)

res = all_dict['results'].to_dict('records')
file_p = f"data/all_results.json"
with open(file_p, "w") as final:
    json.dump(res, final)


annotation_file = 'data/all_anno.json'
results_file = 'data/all_results.json'

# create coco object and coco_result object
coco = COCO(annotation_file)
coco_result = coco.loadRes(results_file)

# create coco_eval object by taking coco and coco_result
coco_eval = COCOEvalCap(coco, coco_result)

# evaluate on a subset of images by setting
# coco_eval.params['image_id'] = coco_result.getImgIds()
# please remove this line when evaluating the full validation set
coco_eval.params['image_id'] = coco_result.getImgIds()

# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
coco_eval.evaluate()

/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying t

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 4917, 'reflen': 7910, 'guess': [4917, 4360, 3803, 3246], 'correct': [1347, 346, 119, 51]}
ratio: 0.6216182048039669
Bleu_1: 0.149
Bleu_2: 0.080
Bleu_3: 0.048
Bleu_4: 0.031
computing METEOR score...
METEOR: 0.078
computing Rouge score...
ROUGE_L: 0.178
computing CIDEr score...
CIDEr: 0.385
computing SPICE score...
SPICE: 0.119


In [19]:
file_dict_f = file_path_dict(pg_dict_f, test='fat')
web_res_dict_f = dict()

for el in pg_dict_f:
    print(el)
    web_res_dict_f[el] = calc_metrics(file_dict_f, topic=el, anno='o')


all_res_f = []
for el in pg_dict_f:
    all_res_f.append(web_res_dict_f[el])

df_res_f = pd.DataFrame(all_res_f)
display(df_res_f.describe().round(2))
#     print(el)
# #     print(split_res_dict[el])
#     for metric, score in web_res_dict_f[el].items():
#         print(f'{metric}: {score:.3f}')
#     print()
all_topics_f = []
for k in web_res_dict_f:
    d_temp = copy.deepcopy(web_res_dict_f[k])
    del d_temp['Bleu_4']
    del d_temp['Bleu_2']
    del d_temp['Bleu_3']
    d_temp['topic'] = k
    all_topics_f.append(d_temp)
#     break
# d_temp
df_tmp_f = pd.DataFrame(all_topics_f)
display(df_tmp_f.round(2))

a
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 442, 'reflen': 766, 'guess': [442, 391, 340, 289], 'correct': [102, 19, 4, 2]}
ratio: 0.5770234986937637
Bleu_1: 0.111
Bleu_2: 0.051
Bleu_3: 0.024
Bleu_4: 0.015
computing METEOR score...
METEOR: 0.055
computing Rouge score...
ROUGE_L: 0.137
computing CIDEr score...
CIDEr: 0.170
computing SPICE score...
SPICE: 0.098
b
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 839, 'reflen': 1454, 'guess': [839, 736, 633, 530], 'correct': [242, 47, 7, 3]}
ratio: 0.5770288858317902
Bleu_1: 0.139
Bleu_2: 0.065
Bleu_3: 0.028
Bleu_4: 0.016
computing METEOR score...
METEOR: 0.078
c

METEOR: 0.081
computing Rouge score...
ROUGE_L: 0.126
computing CIDEr score...
CIDEr: 0.332
computing SPICE score...
SPICE: 0.081


,Bleu_1,Bleu_2,Bleu_3,Bleu_4,METEOR,ROUGE_L,CIDEr,SPICE
count,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.00
mean,0.14,0.07,0.04,0.02,0.07,0.16,0.32,0.10
std,0.04,0.03,0.02,0.02,0.02,0.05,0.22,0.05
min,0.06,0.00,0.00,0.00,0.02,0.07,0.13,0.05
25%,0.11,0.05,0.02,0.01,0.05,0.13,0.18,0.07
50%,0.14,0.07,0.04,0.02,0.07,0.15,0.26,0.10
75%,0.17,0.09,0.05,0.03,0.09,0.20,0.39,0.13
max,0.19,0.11,0.08,0.07,0.10,0.25,1.01,0.21


,Bleu_1,METEOR,ROUGE_L,CIDEr,SPICE,topic
0,0.11,0.05,0.14,0.17,0.10,a
1,0.14,0.08,0.18,0.33,0.11,b
2,0.17,0.10,0.21,0.43,0.16,c
3,0.14,0.09,0.22,0.26,0.14,d
4,0.09,0.06,0.14,0.13,0.09,e
5,0.16,0.07,0.14,0.43,0.06,f
6,0.15,0.06,0.15,0.22,0.07,g
7,0.17,0.09,0.19,0.17,0.10,h
8,0.18,0.10,0.25,1.01,0.21,i
9,0.06,0.02,0.07,0.13,0.07,j


# WIT

In [20]:
df_w1, pg_dict_w = create_dict(df_w)

all_dict = dict()
all_dict['data'] = copy.deepcopy(df_w1)
all_dict['data'] = all_dict['data'].reset_index(drop=True)
temp = all_dict['data'][['image_id', 'ofa_caption']]
temp.columns = temp.columns.str.replace('ofa_caption', 'caption')
all_dict['results'] = temp
all_dict['images'] = id2imdf(all_dict['data'])
all_dict['annotations'] = web_2anno(all_dict['data'])
t_data = dict()

t_data['info'] = jdata['info']
t_data['images'] = all_dict['images'].to_dict('records')
t_data['type'] = jdata['type']
t_data['licenses'] = jdata['licenses']
t_data['annotations'] = all_dict['annotations'].to_dict('records')
file_p = f"data/all_anno.json"
with open(file_p, "w") as final:
    json.dump(t_data, final)
res = all_dict['results'].to_dict('records')
file_p = f"data/all_results.json"
with open(file_p, "w") as final:
    json.dump(res, final)
annotation_file = 'data/all_anno.json'
results_file = 'data/all_results.json'

# create coco object and coco_result object
coco = COCO(annotation_file)
coco_result = coco.loadRes(results_file)

# create coco_eval object by taking coco and coco_result
coco_eval = COCOEvalCap(coco, coco_result)

# evaluate on a subset of images by setting
# coco_eval.params['image_id'] = coco_result.getImgIds()
# please remove this line when evaluating the full validation set
coco_eval.params['image_id'] = coco_result.getImgIds()

# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
coco_eval.evaluate()

/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying t

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 5853, 'reflen': 7910, 'guess': [5853, 5296, 4739, 4183], 'correct': [1190, 265, 87, 30]}
ratio: 0.73994943109978
Bleu_1: 0.143
Bleu_2: 0.071
Bleu_3: 0.040
Bleu_4: 0.024
computing METEOR score...
METEOR: 0.064
computing Rouge score...
ROUGE_L: 0.148
computing CIDEr score...
CIDEr: 0.215
computing SPICE score...
SPICE: 0.100


In [21]:
file_dict_w = file_path_dict(pg_dict_w, test='wit')
web_res_dict_w = dict()

for el in pg_dict_w:
    print(el)
    web_res_dict_w[el] = calc_metrics(file_dict_w, topic=el, anno='o')


all_res_w = []
for el in pg_dict_w:
    all_res_w.append(web_res_dict_w[el])


df_res_w = pd.DataFrame(all_res_w)
display(df_res_w.describe().round(2))
# for el in pg_dict_f:
#     print(el)
# #     print(split_res_dict[el])
#     for metric, score in web_res_dict_f[el].items():
#         print(f'{metric}: {score:.3f}')
#     print()
all_topics_w = []
for k in web_res_dict_w:
    d_temp = copy.deepcopy(web_res_dict_w[k])
    del d_temp['Bleu_4']
    del d_temp['Bleu_2']
    del d_temp['Bleu_3']
    d_temp['topic'] = k
    all_topics_w.append(d_temp)
#     break
# d_temp
df_tmp_w = pd.DataFrame(all_topics_w)
display(df_tmp_w.round(2))

a
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 550, 'reflen': 766, 'guess': [550, 499, 448, 397], 'correct': [90, 13, 4, 0]}
ratio: 0.7180156657954073
Bleu_1: 0.110
Bleu_2: 0.044
Bleu_3: 0.023
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.044
computing Rouge score...
ROUGE_L: 0.112
computing CIDEr score...
CIDEr: 0.160
computing SPICE score...
SPICE: 0.059
b
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 1100, 'reflen': 1454, 'guess': [1100, 997, 894, 791], 'correct': [215, 32, 5, 2]}
ratio: 0.7565337001370313
Bleu_1: 0.142
Bleu_2: 0.057
Bleu_3: 0.024
Bleu_4: 0.013
computing METEOR score...
METEOR: 0.064


METEOR: 0.069
computing Rouge score...
ROUGE_L: 0.122
computing CIDEr score...
CIDEr: 0.188
computing SPICE score...
SPICE: 0.099


,Bleu_1,Bleu_2,Bleu_3,Bleu_4,METEOR,ROUGE_L,CIDEr,SPICE
count,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.00
mean,0.13,0.06,0.03,0.01,0.06,0.14,0.19,0.09
std,0.03,0.03,0.02,0.02,0.02,0.04,0.07,0.04
min,0.07,0.00,0.00,0.00,0.03,0.08,0.06,0.04
25%,0.11,0.04,0.02,0.00,0.04,0.11,0.14,0.06
50%,0.12,0.06,0.03,0.01,0.05,0.12,0.19,0.08
75%,0.15,0.08,0.04,0.02,0.07,0.16,0.25,0.12
max,0.18,0.10,0.06,0.04,0.09,0.21,0.30,0.17


,Bleu_1,METEOR,ROUGE_L,CIDEr,SPICE,topic
0,0.11,0.04,0.11,0.16,0.06,a
1,0.14,0.06,0.14,0.19,0.08,b
2,0.15,0.08,0.17,0.30,0.14,c
3,0.13,0.08,0.21,0.25,0.17,d
4,0.11,0.05,0.12,0.12,0.06,e
5,0.11,0.04,0.09,0.06,0.08,f
6,0.11,0.04,0.11,0.13,0.04,g
7,0.16,0.07,0.15,0.18,0.08,h
8,0.17,0.08,0.19,0.29,0.14,i
9,0.07,0.03,0.08,0.10,0.04,j


# Random

In [22]:
df_r1, pg_dict_r = create_dict(df_r)

all_dict = dict()
all_dict['data'] = copy.deepcopy(df_r1)
all_dict['data'] = all_dict['data'].reset_index(drop=True)
temp = all_dict['data'][['image_id', 'ofa_caption']]
temp.columns = temp.columns.str.replace('ofa_caption', 'caption')
all_dict['results'] = temp
all_dict['images'] = id2imdf(all_dict['data'])
all_dict['annotations'] = web_2anno(all_dict['data'])
t_data = dict()

t_data['info'] = jdata['info']
t_data['images'] = all_dict['images'].to_dict('records')
t_data['type'] = jdata['type']
t_data['licenses'] = jdata['licenses']
t_data['annotations'] = all_dict['annotations'].to_dict('records')
file_p = f"data/all_anno.json"
with open(file_p, "w") as final:
    json.dump(t_data, final)
res = all_dict['results'].to_dict('records')
file_p = f"data/all_results.json"
with open(file_p, "w") as final:
    json.dump(res, final)
annotation_file = 'data/all_anno.json'
results_file = 'data/all_results.json'

# create coco object and coco_result object
coco = COCO(annotation_file)
coco_result = coco.loadRes(results_file)

# create coco_eval object by taking coco and coco_result
coco_eval = COCOEvalCap(coco, coco_result)

# evaluate on a subset of images by setting
# coco_eval.params['image_id'] = coco_result.getImgIds()
# please remove this line when evaluating the full validation set
coco_eval.params['image_id'] = coco_result.getImgIds()

# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
coco_eval.evaluate()

/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying t

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 4504, 'reflen': 7910, 'guess': [4504, 3947, 3390, 2833], 'correct': [964, 201, 57, 19]}
ratio: 0.5694058154234425
Bleu_1: 0.100
Bleu_2: 0.049
Bleu_3: 0.027
Bleu_4: 0.016
computing METEOR score...
METEOR: 0.053
computing Rouge score...
ROUGE_L: 0.133
computing CIDEr score...
CIDEr: 0.189
computing SPICE score...
SPICE: 0.093


In [23]:
file_dict_r = file_path_dict(pg_dict_r, test='ran')
web_res_dict_r = dict()

for el in pg_dict_r:
    print(el)
    web_res_dict_r[el] = calc_metrics(file_dict_r, topic=el, anno='o')


all_res_r = []
for el in pg_dict_r:
    all_res_r.append(web_res_dict_r[el])


df_res_r = pd.DataFrame(all_res_r)
display(df_res_r.describe().round(2))
# for el in pg_dict_f:
#     print(el)
# #     print(split_res_dict[el])
#     for metric, score in web_res_dict_f[el].items():
#         print(f'{metric}: {score:.3f}')
#     print()
all_topics_r = []
for k in web_res_dict_r:
    d_temp = copy.deepcopy(web_res_dict_r[k])
    del d_temp['Bleu_4']
    del d_temp['Bleu_2']
    del d_temp['Bleu_3']
    d_temp['topic'] = k
    all_topics_r.append(d_temp)
#     break
# d_temp
df_tmp_r = pd.DataFrame(all_topics_r)
display(df_tmp_r.round(2))

a
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 415, 'reflen': 766, 'guess': [415, 364, 313, 262], 'correct': [82, 9, 2, 1]}
ratio: 0.5417754569183528
Bleu_1: 0.085
Bleu_2: 0.030
Bleu_3: 0.014
Bleu_4: 0.008
computing METEOR score...
METEOR: 0.043
computing Rouge score...
ROUGE_L: 0.108
computing CIDEr score...
CIDEr: 0.138
computing SPICE score...
SPICE: 0.074
b
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 796, 'reflen': 1454, 'guess': [796, 693, 590, 487], 'correct': [176, 27, 5, 0]}
ratio: 0.5474552957355244
Bleu_1: 0.097
Bleu_2: 0.041
Bleu_3: 0.018
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.055
com

METEOR: 0.052
computing Rouge score...
ROUGE_L: 0.115
computing CIDEr score...
CIDEr: 0.257
computing SPICE score...
SPICE: 0.073


,Bleu_1,Bleu_2,Bleu_3,Bleu_4,METEOR,ROUGE_L,CIDEr,SPICE
count,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.00
mean,0.09,0.04,0.02,0.01,0.05,0.13,0.20,0.08
std,0.03,0.02,0.02,0.01,0.02,0.03,0.06,0.04
min,0.04,0.00,0.00,0.00,0.01,0.08,0.11,0.00
25%,0.08,0.03,0.01,0.00,0.03,0.10,0.14,0.06
50%,0.10,0.05,0.02,0.01,0.05,0.13,0.20,0.07
75%,0.11,0.06,0.03,0.02,0.06,0.15,0.25,0.11
max,0.14,0.08,0.05,0.03,0.08,0.19,0.29,0.15


,Bleu_1,METEOR,ROUGE_L,CIDEr,SPICE,topic
0,0.08,0.04,0.11,0.14,0.07,a
1,0.10,0.05,0.14,0.18,0.09,b
2,0.10,0.06,0.16,0.19,0.15,c
3,0.09,0.07,0.18,0.20,0.14,d
4,0.04,0.03,0.09,0.11,0.06,e
5,0.11,0.04,0.10,0.28,0.01,f
6,0.12,0.05,0.13,0.21,0.09,g
7,0.10,0.06,0.14,0.14,0.07,h
8,0.13,0.07,0.17,0.24,0.13,i
9,0.08,0.01,0.09,0.24,0.00,j


In [44]:
base = df_tmp[['SPICE']]
base.rename(columns={'SPICE': 'Base'}, inplace=True)

fat = df_tmp_f[['SPICE']]
fat.rename(columns={'SPICE': 'App_2'}, inplace=True)

wit = df_tmp_w[['SPICE']]
wit.rename(columns={'SPICE': 'App_1'}, inplace=True)

ran = df_tmp_r[['SPICE']]
ran.rename(columns={'SPICE': 'Random'}, inplace=True)

t = df_tmp_r[['topic']]

/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [45]:
vertical_concat = pd.concat([t, wit, fat, base, ran], axis=1)
vertical_concat

,topic,App_1,App_2,Base,Random
0,a,0.058934,0.097687,0.074795,0.074260
1,b,0.084542,0.108382,0.084199,0.086676
2,c,0.138019,0.158847,0.141086,0.149090
3,d,0.168552,0.144040,0.115619,0.135432
4,e,0.063840,0.088897,0.065591,0.064086
5,f,0.080166,0.056994,0.042421,0.008065
6,g,0.041796,0.071570,0.052911,0.087815
7,h,0.077287,0.098291,0.063167,0.070070
8,i,0.135723,0.208502,0.092757,0.125488
9,j,0.036905,0.069629,0.000000,0.000000


In [46]:
vertical_concat.to_csv(f'ped_spice.csv', index=False)

# BASE human ped

In [26]:
# gk = df_wp.groupby('topics')
# pg_dict = dict()
# for el in gk.groups.keys():
all_dict = dict()
all_dict['data'] = copy.deepcopy(df_n)
all_dict['data'] = all_dict['data'].reset_index(drop=True)
temp = all_dict['data'][['image_id', 'ofa_caption']]
temp.columns = temp.columns.str.replace('ofa_caption', 'caption')
all_dict['results'] = temp
all_dict['images'] = id2imdf(all_dict['data'])
all_dict['human_annotations'] = h_gt2anno(all_dict['data'])

/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying t

In [27]:
t_data = dict()
t_data['info'] = jdata['info']
t_data['images'] = all_dict['images'].to_dict('records')
t_data['type'] = jdata['type']
t_data['licenses'] = jdata['licenses']
t_data['annotations'] = all_dict['human_annotations'].to_dict('records')
file_p = f"data/all_anno.json"
with open(file_p, "w") as final:
    json.dump(t_data, final)

In [28]:
res = all_dict['results'].to_dict('records')
file_p = f"data/all_results.json"
with open(file_p, "w") as final:
    json.dump(res, final)

In [29]:
annotation_file = 'data/all_anno.json'
results_file = 'data/all_results.json'

# create coco object and coco_result object
coco = COCO(annotation_file)
coco_result = coco.loadRes(results_file)

# create coco_eval object by taking coco and coco_result
coco_eval = COCOEvalCap(coco, coco_result)

# evaluate on a subset of images by setting
# coco_eval.params['image_id'] = coco_result.getImgIds()
# please remove this line when evaluating the full validation set
coco_eval.params['image_id'] = coco_result.getImgIds()

# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
coco_eval.evaluate()

# # print output evaluation scores
# for metric, score in coco_eval.eval.items():
#     print(f'{metric}: {score:.3f}')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 5370, 'reflen': 3219, 'guess': [5370, 4813, 4256, 3699], 'correct': [1364, 353, 85, 22]}
ratio: 1.6682199440814949
Bleu_1: 0.254
Bleu_2: 0.136
Bleu_3: 0.072
Bleu_4: 0.039
computing METEOR score...
METEOR: 0.165
computing Rouge score...
ROUGE_L: 0.286
computing CIDEr score...
CIDEr: 0.497
computing SPICE score...
SPICE: 0.187


In [30]:
human_res_dict = dict()
for el in gk.groups.keys():
    human_res_dict[el] = calc_metrics(topic=el, anno='s')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 340, 'reflen': 276, 'guess': [340, 305, 270, 235], 'correct': [117, 28, 4, 1]}
ratio: 1.231884057966551
Bleu_1: 0.344
Bleu_2: 0.178
Bleu_3: 0.078
Bleu_4: 0.038
computing METEOR score...
METEOR: 0.167
computing Rouge score...
ROUGE_L: 0.327
computing CIDEr score...
CIDEr: 0.659
computing SPICE score...
SPICE: 0.208
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 853, 'reflen': 512, 'guess': [853, 763, 673, 583], 'correct': [225, 50, 12, 4]}
ratio: 1.6660156249967462
Bleu_1: 0.264
Bleu_2: 0.131
Bleu_3: 0.068
Bleu_4: 0.038
computing METEOR score...
METEOR: 0.168
comput

SPICE: 0.178


In [31]:
h_res = []
for el in gk.groups.keys():
    h_res.append(human_res_dict[el])
df_res = pd.DataFrame(h_res)
df_res.describe().round(2)

,Bleu_1,Bleu_2,Bleu_3,Bleu_4,METEOR,ROUGE_L,CIDEr,SPICE
count,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.00
mean,0.24,0.12,0.06,0.03,0.15,0.27,0.45,0.17
std,0.09,0.06,0.05,0.03,0.04,0.07,0.18,0.05
min,0.05,0.00,0.00,0.00,0.07,0.09,0.00,0.07
25%,0.20,0.09,0.02,0.00,0.12,0.23,0.36,0.14
50%,0.22,0.12,0.06,0.03,0.16,0.27,0.45,0.18
75%,0.29,0.16,0.09,0.05,0.17,0.32,0.55,0.21
max,0.39,0.26,0.17,0.10,0.22,0.38,0.80,0.29


In [32]:
for el in gk.groups.keys():
    print(el)
#     print(split_res_dict[el])
    for metric, score in human_res_dict[el].items():
        print(f'{metric}: {score:.3f}')
    print()

a
Bleu_1: 0.344
Bleu_2: 0.178
Bleu_3: 0.078
Bleu_4: 0.038
METEOR: 0.167
ROUGE_L: 0.327
CIDEr: 0.659
SPICE: 0.208

b
Bleu_1: 0.264
Bleu_2: 0.131
Bleu_3: 0.068
Bleu_4: 0.038
METEOR: 0.168
ROUGE_L: 0.309
CIDEr: 0.478
SPICE: 0.186

c
Bleu_1: 0.209
Bleu_2: 0.105
Bleu_3: 0.043
Bleu_4: 0.000
METEOR: 0.169
ROUGE_L: 0.250
CIDEr: 0.414
SPICE: 0.204

d
Bleu_1: 0.167
Bleu_2: 0.077
Bleu_3: 0.031
Bleu_4: 0.000
METEOR: 0.189
ROUGE_L: 0.234
CIDEr: 0.290
SPICE: 0.215

e
Bleu_1: 0.312
Bleu_2: 0.153
Bleu_3: 0.082
Bleu_4: 0.037
METEOR: 0.191
ROUGE_L: 0.337
CIDEr: 0.451
SPICE: 0.203

f
Bleu_1: 0.294
Bleu_2: 0.154
Bleu_3: 0.092
Bleu_4: 0.062
METEOR: 0.122
ROUGE_L: 0.299
CIDEr: 0.456
SPICE: 0.162

g
Bleu_1: 0.139
Bleu_2: 0.073
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.114
ROUGE_L: 0.190
CIDEr: 0.310
SPICE: 0.072

h
Bleu_1: 0.184
Bleu_2: 0.093
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.134
ROUGE_L: 0.235
CIDEr: 0.295
SPICE: 0.139

i
Bleu_1: 0.281
Bleu_2: 0.165
Bleu_3: 0.093
Bleu_4: 0.052
METEOR: 0.157
ROUGE_L: 0.303
C

In [33]:
all_topics = []
for k in human_res_dict:
    d_temp = copy.deepcopy(human_res_dict[k])
    del d_temp['Bleu_4']
    del d_temp['Bleu_2']
    del d_temp['Bleu_3']
    d_temp['topic'] = k
    all_topics.append(d_temp)
#     break
# d_temp
df_tmp = pd.DataFrame(all_topics)
df_tmp.round(2)

,Bleu_1,METEOR,ROUGE_L,CIDEr,SPICE,topic
0,0.34,0.17,0.33,0.66,0.21,a
1,0.26,0.17,0.31,0.48,0.19,b
2,0.21,0.17,0.25,0.41,0.20,c
3,0.17,0.19,0.23,0.29,0.21,d
4,0.31,0.19,0.34,0.45,0.20,e
5,0.29,0.12,0.30,0.46,0.16,f
6,0.14,0.11,0.19,0.31,0.07,g
7,0.18,0.13,0.23,0.30,0.14,h
8,0.28,0.16,0.30,0.54,0.15,i
9,0.05,0.07,0.09,0.00,0.10,j


# base Ped web

In [34]:
# gk = df_wp.groupby('topics')
# pg_dict = dict()
# for el in gk.groups.keys():
all_dict = dict()
all_dict['data'] = copy.deepcopy(df_n)
all_dict['data'] = all_dict['data'].reset_index(drop=True)
temp = all_dict['data'][['image_id', 'ofa_caption']]
temp.columns = temp.columns.str.replace('ofa_caption', 'caption')
all_dict['results'] = temp
all_dict['images'] = id2imdf(all_dict['data'])
all_dict['human_annotations'] = web_2anno(all_dict['data'])

/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying t

In [35]:
t_data = dict()
t_data['info'] = jdata['info']
t_data['images'] = all_dict['images'].to_dict('records')
t_data['type'] = jdata['type']
t_data['licenses'] = jdata['licenses']
t_data['annotations'] = all_dict['human_annotations'].to_dict('records')
file_p = f"data/all_anno.json"
with open(file_p, "w") as final:
    json.dump(t_data, final)

In [36]:
res = all_dict['results'].to_dict('records')
file_p = f"data/all_results.json"
with open(file_p, "w") as final:
    json.dump(res, final)

In [37]:
annotation_file = 'data/all_anno.json'
results_file = 'data/all_results.json'

# create coco object and coco_result object
coco = COCO(annotation_file)
coco_result = coco.loadRes(results_file)

# create coco_eval object by taking coco and coco_result
coco_eval = COCOEvalCap(coco, coco_result)

# evaluate on a subset of images by setting
# coco_eval.params['image_id'] = coco_result.getImgIds()
# please remove this line when evaluating the full validation set
coco_eval.params['image_id'] = coco_result.getImgIds()

# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
coco_eval.evaluate()

# # print output evaluation scores
# for metric, score in coco_eval.eval.items():
#     print(f'{metric}: {score:.3f}')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 5370, 'reflen': 7910, 'guess': [5370, 4813, 4256, 3699], 'correct': [1068, 181, 45, 15]}
ratio: 0.6788874841971329
Bleu_1: 0.124
Bleu_2: 0.054
Bleu_3: 0.027
Bleu_4: 0.015
computing METEOR score...
METEOR: 0.059
computing Rouge score...
ROUGE_L: 0.133
computing CIDEr score...
CIDEr: 0.216
computing SPICE score...
SPICE: 0.088


In [38]:
web_res_dict = dict()
for el in gk.groups.keys():
#     split_res_dict[el] = calc_metrics(topic=el, anno='s')
#     con_res_dict[el] = calc_metrics(topic=el, anno='c')
    web_res_dict[el] = calc_metrics(topic=el, anno='o')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 340, 'reflen': 466, 'guess': [340, 305, 270, 235], 'correct': [57, 10, 2, 0]}
ratio: 0.7296137339040137
Bleu_1: 0.116
Bleu_2: 0.051
Bleu_3: 0.024
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.047
computing Rouge score...
ROUGE_L: 0.112
computing CIDEr score...
CIDEr: 0.173
computing SPICE score...
SPICE: 0.091
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 853, 'reflen': 1242, 'guess': [853, 763, 673, 583], 'correct': [169, 25, 3, 2]}
ratio: 0.6867954911427643
Bleu_1: 0.126
Bleu_2: 0.051
Bleu_3: 0.019
Bleu_4: 0.011
computing METEOR score...
METEOR: 0.064
comput

SPICE: 0.095


In [39]:
all_res = []
for el in gk.groups.keys():
    all_res.append(web_res_dict[el])
df_res = pd.DataFrame(all_res)
df_res.describe().round(2)

,Bleu_1,Bleu_2,Bleu_3,Bleu_4,METEOR,ROUGE_L,CIDEr,SPICE
count,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.00
mean,0.11,0.04,0.02,0.01,0.05,0.12,0.18,0.07
std,0.02,0.02,0.01,0.01,0.02,0.04,0.06,0.04
min,0.06,0.00,0.00,0.00,0.01,0.06,0.10,0.00
25%,0.10,0.03,0.01,0.00,0.03,0.10,0.13,0.04
50%,0.10,0.04,0.02,0.00,0.05,0.11,0.17,0.07
75%,0.12,0.05,0.03,0.01,0.07,0.14,0.24,0.09
max,0.15,0.08,0.04,0.02,0.09,0.21,0.29,0.15


In [40]:
for el in gk.groups.keys():
    print(el)
#     print(split_res_dict[el])
    for metric, score in web_res_dict[el].items():
        print(f'{metric}: {score:.3f}')
    print()

a
Bleu_1: 0.116
Bleu_2: 0.051
Bleu_3: 0.024
Bleu_4: 0.000
METEOR: 0.047
ROUGE_L: 0.112
CIDEr: 0.173
SPICE: 0.091

b
Bleu_1: 0.126
Bleu_2: 0.051
Bleu_3: 0.019
Bleu_4: 0.011
METEOR: 0.064
ROUGE_L: 0.127
CIDEr: 0.248
SPICE: 0.083

c
Bleu_1: 0.135
Bleu_2: 0.069
Bleu_3: 0.039
Bleu_4: 0.023
METEOR: 0.068
ROUGE_L: 0.154
CIDEr: 0.232
SPICE: 0.140

d
Bleu_1: 0.103
Bleu_2: 0.045
Bleu_3: 0.022
Bleu_4: 0.011
METEOR: 0.067
ROUGE_L: 0.153
CIDEr: 0.129
SPICE: 0.108

e
Bleu_1: 0.102
Bleu_2: 0.033
Bleu_3: 0.017
Bleu_4: 0.009
METEOR: 0.050
ROUGE_L: 0.118
CIDEr: 0.099
SPICE: 0.069

f
Bleu_1: 0.097
Bleu_2: 0.033
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.032
ROUGE_L: 0.096
CIDEr: 0.149
SPICE: 0.042

g
Bleu_1: 0.102
Bleu_2: 0.022
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.035
ROUGE_L: 0.101
CIDEr: 0.131
SPICE: 0.053

h
Bleu_1: 0.115
Bleu_2: 0.034
Bleu_3: 0.015
Bleu_4: 0.000
METEOR: 0.059
ROUGE_L: 0.124
CIDEr: 0.110
SPICE: 0.065

i
Bleu_1: 0.140
Bleu_2: 0.065
Bleu_3: 0.037
Bleu_4: 0.024
METEOR: 0.067
ROUGE_L: 0.159
C

In [41]:
all_topics = []
for k in web_res_dict:
    d_temp = copy.deepcopy(web_res_dict[k])
    del d_temp['Bleu_4']
    del d_temp['Bleu_2']
    del d_temp['Bleu_3']
    d_temp['topic'] = k
    all_topics.append(d_temp)
#     break
# d_temp
df_tmp = pd.DataFrame(all_topics)
df_tmp.round(2)

,Bleu_1,METEOR,ROUGE_L,CIDEr,SPICE,topic
0,0.12,0.05,0.11,0.17,0.09,a
1,0.13,0.06,0.13,0.25,0.08,b
2,0.14,0.07,0.15,0.23,0.14,c
3,0.10,0.07,0.15,0.13,0.11,d
4,0.10,0.05,0.12,0.10,0.07,e
5,0.10,0.03,0.10,0.15,0.04,f
6,0.10,0.04,0.10,0.13,0.05,g
7,0.11,0.06,0.12,0.11,0.07,h
8,0.14,0.07,0.16,0.25,0.09,i
9,0.07,0.01,0.06,0.14,0.00,j


# ped WIT human


In [27]:
# gk = df_wp.groupby('topics')
# pg_dict = dict()
# for el in gk.groups.keys():
all_dict = dict()
all_dict['data'] = copy.deepcopy(df_n)
all_dict['data'] = all_dict['data'].reset_index(drop=True)
temp = all_dict['data'][['image_id', 'ofa_caption']]
temp.columns = temp.columns.str.replace('ofa_caption', 'caption')
all_dict['results'] = temp
all_dict['images'] = id2imdf(all_dict['data'])
all_dict['human_annotations'] = h_gt2anno(all_dict['data'])

t_data = dict()
t_data['info'] = jdata['info']
t_data['images'] = all_dict['images'].to_dict('records')
t_data['type'] = jdata['type']
t_data['licenses'] = jdata['licenses']
t_data['annotations'] = all_dict['human_annotations'].to_dict('records')
file_p = f"data/all_anno.json"
with open(file_p, "w") as final:
    json.dump(t_data, final)
    
res = all_dict['results'].to_dict('records')
file_p = f"data/all_results.json"
with open(file_p, "w") as final:
    json.dump(res, final)

    
annotation_file = 'data/all_anno.json'
results_file = 'data/all_results.json'

# create coco object and coco_result object
coco = COCO(annotation_file)
coco_result = coco.loadRes(results_file)

# create coco_eval object by taking coco and coco_result
coco_eval = COCOEvalCap(coco, coco_result)

# evaluate on a subset of images by setting
# coco_eval.params['image_id'] = coco_result.getImgIds()
# please remove this line when evaluating the full validation set
coco_eval.params['image_id'] = coco_result.getImgIds()

# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
coco_eval.evaluate()

# # print output evaluation scores
# for metric, score in coco_eval.eval.items():
#     print(f'{metric}: {score:.3f}')

/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying t

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 5445, 'reflen': 3219, 'guess': [5445, 4888, 4331, 3775], 'correct': [1205, 250, 76, 21]}
ratio: 1.6915191053116834
Bleu_1: 0.221
Bleu_2: 0.106
Bleu_3: 0.058
Bleu_4: 0.032
computing METEOR score...
METEOR: 0.146
computing Rouge score...
ROUGE_L: 0.242
computing CIDEr score...
CIDEr: 0.414
computing SPICE score...
SPICE: 0.160


In [28]:
human_res_dict = dict()
for el in gk.groups.keys():
    human_res_dict[el] = calc_metrics(topic=el, anno='s')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 348, 'reflen': 276, 'guess': [348, 313, 278, 243], 'correct': [88, 17, 5, 1]}
ratio: 1.260869565212823
Bleu_1: 0.253
Bleu_2: 0.117
Bleu_3: 0.063
Bleu_4: 0.032
computing METEOR score...
METEOR: 0.118
computing Rouge score...
ROUGE_L: 0.248
computing CIDEr score...
CIDEr: 0.387
computing SPICE score...
SPICE: 0.124
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 892, 'reflen': 512, 'guess': [892, 802, 712, 622], 'correct': [195, 27, 7, 2]}
ratio: 1.7421874999965974
Bleu_1: 0.219
Bleu_2: 0.086
Bleu_3: 0.042
Bleu_4: 0.022
computing METEOR score...
METEOR: 0.137
computin

SPICE: 0.144


In [29]:
h_res = []
for el in gk.groups.keys():
    h_res.append(human_res_dict[el])
df_res = pd.DataFrame(h_res)
df_res.describe().round(2)

,Bleu_1,Bleu_2,Bleu_3,Bleu_4,METEOR,ROUGE_L,CIDEr,SPICE
count,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.00
mean,0.21,0.10,0.05,0.02,0.14,0.23,0.37,0.15
std,0.08,0.06,0.04,0.03,0.04,0.07,0.19,0.05
min,0.03,0.00,0.00,0.00,0.04,0.05,0.00,0.08
25%,0.18,0.07,0.03,0.00,0.11,0.20,0.26,0.13
50%,0.20,0.09,0.04,0.02,0.14,0.23,0.38,0.13
75%,0.23,0.11,0.06,0.03,0.15,0.25,0.43,0.16
max,0.40,0.25,0.16,0.10,0.23,0.37,0.83,0.30


In [30]:
for el in gk.groups.keys():
    print(el)
#     print(split_res_dict[el])
    for metric, score in human_res_dict[el].items():
        print(f'{metric}: {score:.3f}')
    print()

a
Bleu_1: 0.253
Bleu_2: 0.117
Bleu_3: 0.063
Bleu_4: 0.032
METEOR: 0.118
ROUGE_L: 0.248
CIDEr: 0.387
SPICE: 0.124

b
Bleu_1: 0.219
Bleu_2: 0.086
Bleu_3: 0.042
Bleu_4: 0.022
METEOR: 0.137
ROUGE_L: 0.249
CIDEr: 0.268
SPICE: 0.167

c
Bleu_1: 0.177
Bleu_2: 0.067
Bleu_3: 0.028
Bleu_4: 0.016
METEOR: 0.154
ROUGE_L: 0.185
CIDEr: 0.282
SPICE: 0.130

d
Bleu_1: 0.153
Bleu_2: 0.057
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.172
ROUGE_L: 0.206
CIDEr: 0.235
SPICE: 0.204

e
Bleu_1: 0.212
Bleu_2: 0.070
Bleu_3: 0.031
Bleu_4: 0.000
METEOR: 0.117
ROUGE_L: 0.227
CIDEr: 0.233
SPICE: 0.133

f
Bleu_1: 0.196
Bleu_2: 0.060
Bleu_3: 0.029
Bleu_4: 0.000
METEOR: 0.101
ROUGE_L: 0.213
CIDEr: 0.293
SPICE: 0.131

g
Bleu_1: 0.190
Bleu_2: 0.086
Bleu_3: 0.040
Bleu_4: 0.000
METEOR: 0.133
ROUGE_L: 0.203
CIDEr: 0.378
SPICE: 0.103

h
Bleu_1: 0.197
Bleu_2: 0.099
Bleu_3: 0.055
Bleu_4: 0.038
METEOR: 0.139
ROUGE_L: 0.240
CIDEr: 0.410
SPICE: 0.130

i
Bleu_1: 0.264
Bleu_2: 0.153
Bleu_3: 0.091
Bleu_4: 0.050
METEOR: 0.154
ROUGE_L: 0.275
C

In [31]:
all_topics = []
for k in human_res_dict:
    d_temp = copy.deepcopy(human_res_dict[k])
    del d_temp['Bleu_4']
    del d_temp['Bleu_2']
    del d_temp['Bleu_3']
    d_temp['topic'] = k
    all_topics.append(d_temp)
#     break
# d_temp
df_tmp = pd.DataFrame(all_topics)
df_tmp.round(2)

,Bleu_1,METEOR,ROUGE_L,CIDEr,SPICE,topic
0,0.25,0.12,0.25,0.39,0.12,a
1,0.22,0.14,0.25,0.27,0.17,b
2,0.18,0.15,0.19,0.28,0.13,c
3,0.15,0.17,0.21,0.24,0.20,d
4,0.21,0.12,0.23,0.23,0.13,e
5,0.20,0.10,0.21,0.29,0.13,f
6,0.19,0.13,0.20,0.38,0.10,g
7,0.20,0.14,0.24,0.41,0.13,h
8,0.26,0.15,0.28,0.52,0.15,i
9,0.03,0.04,0.05,0.00,0.08,j


# wit ped  web

In [29]:
# gk = df_wp.groupby('topics')
# pg_dict = dict()
# for el in gk.groups.keys():
all_dict = dict()
all_dict['data'] = copy.deepcopy(df_n)
all_dict['data'] = all_dict['data'].reset_index(drop=True)
temp = all_dict['data'][['image_id', 'ofa_caption']]
temp.columns = temp.columns.str.replace('ofa_caption', 'caption')
all_dict['results'] = temp
all_dict['images'] = id2imdf(all_dict['data'])
all_dict['human_annotations'] = web_2anno(all_dict['data'])

t_data = dict()
t_data['info'] = jdata['info']
t_data['images'] = all_dict['images'].to_dict('records')
t_data['type'] = jdata['type']
t_data['licenses'] = jdata['licenses']
t_data['annotations'] = all_dict['human_annotations'].to_dict('records')
file_p = f"data/all_anno.json"
with open(file_p, "w") as final:
    json.dump(t_data, final)
    
res = all_dict['results'].to_dict('records')
file_p = f"data/all_results.json"
with open(file_p, "w") as final:
    json.dump(res, final)

    
annotation_file = 'data/all_anno.json'
results_file = 'data/all_results.json'

# create coco object and coco_result object
coco = COCO(annotation_file)
coco_result = coco.loadRes(results_file)

# create coco_eval object by taking coco and coco_result
coco_eval = COCOEvalCap(coco, coco_result)

# evaluate on a subset of images by setting
# coco_eval.params['image_id'] = coco_result.getImgIds()
# please remove this line when evaluating the full validation set
coco_eval.params['image_id'] = coco_result.getImgIds()

# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
coco_eval.evaluate()

# # print output evaluation scores
# for metric, score in coco_eval.eval.items():
#     print(f'{metric}: {score:.3f}')

/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying t

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 5012, 'reflen': 7910, 'guess': [5012, 4455, 3898, 3342], 'correct': [1133, 257, 77, 29]}
ratio: 0.6336283185839906
Bleu_1: 0.127
Bleu_2: 0.064
Bleu_3: 0.036
Bleu_4: 0.022
computing METEOR score...
METEOR: 0.063
computing Rouge score...
ROUGE_L: 0.148
computing CIDEr score...
CIDEr: 0.220
computing SPICE score...
SPICE: 0.097


In [30]:
web_res_dict = dict()
for el in gk.groups.keys():
#     split_res_dict[el] = calc_metrics(topic=el, anno='s')
#     con_res_dict[el] = calc_metrics(topic=el, anno='c')
    web_res_dict[el] = calc_metrics(topic=el, anno='o')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 329, 'reflen': 466, 'guess': [329, 294, 259, 224], 'correct': [51, 6, 0, 0]}
ratio: 0.7060085836894721
Bleu_1: 0.102
Bleu_2: 0.037
Bleu_3: 0.000
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.042
computing Rouge score...
ROUGE_L: 0.108
computing CIDEr score...
CIDEr: 0.112
computing SPICE score...
SPICE: 0.060
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 791, 'reflen': 1242, 'guess': [791, 701, 611, 521], 'correct': [175, 29, 4, 0]}
ratio: 0.6368760064407111
Bleu_1: 0.125
Bleu_2: 0.054
Bleu_3: 0.022
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.063
computi

SPICE: 0.098


In [31]:
all_res = []
for el in gk.groups.keys():
    all_res.append(web_res_dict[el])
df_res = pd.DataFrame(all_res)
df_res.describe().round(2)

,Bleu_1,Bleu_2,Bleu_3,Bleu_4,METEOR,ROUGE_L,CIDEr,SPICE
count,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.00
mean,0.11,0.05,0.03,0.02,0.06,0.14,0.23,0.09
std,0.03,0.02,0.02,0.02,0.02,0.04,0.10,0.03
min,0.06,0.00,0.00,0.00,0.02,0.07,0.11,0.05
25%,0.10,0.04,0.02,0.00,0.04,0.12,0.14,0.06
50%,0.11,0.05,0.03,0.02,0.06,0.14,0.22,0.08
75%,0.14,0.07,0.04,0.03,0.07,0.16,0.27,0.11
max,0.16,0.09,0.06,0.05,0.09,0.20,0.41,0.14


In [32]:
for el in gk.groups.keys():
    print(el)
#     print(split_res_dict[el])
    for metric, score in web_res_dict[el].items():
        print(f'{metric}: {score:.3f}')
    print()

a
Bleu_1: 0.102
Bleu_2: 0.037
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.042
ROUGE_L: 0.108
CIDEr: 0.112
SPICE: 0.060

b
Bleu_1: 0.125
Bleu_2: 0.054
Bleu_3: 0.022
Bleu_4: 0.000
METEOR: 0.063
ROUGE_L: 0.136
CIDEr: 0.186
SPICE: 0.083

c
Bleu_1: 0.120
Bleu_2: 0.070
Bleu_3: 0.044
Bleu_4: 0.029
METEOR: 0.068
ROUGE_L: 0.158
CIDEr: 0.202
SPICE: 0.133

d
Bleu_1: 0.105
Bleu_2: 0.055
Bleu_3: 0.021
Bleu_4: 0.000
METEOR: 0.075
ROUGE_L: 0.185
CIDEr: 0.152
SPICE: 0.129

e
Bleu_1: 0.079
Bleu_2: 0.038
Bleu_3: 0.024
Bleu_4: 0.016
METEOR: 0.048
ROUGE_L: 0.120
CIDEr: 0.133
SPICE: 0.064

f
Bleu_1: 0.156
Bleu_2: 0.079
Bleu_3: 0.059
Bleu_4: 0.046
METEOR: 0.066
ROUGE_L: 0.157
CIDEr: 0.410
SPICE: 0.083

g
Bleu_1: 0.115
Bleu_2: 0.046
Bleu_3: 0.023
Bleu_4: 0.000
METEOR: 0.053
ROUGE_L: 0.134
CIDEr: 0.230
SPICE: 0.087

h
Bleu_1: 0.135
Bleu_2: 0.061
Bleu_3: 0.035
Bleu_4: 0.023
METEOR: 0.064
ROUGE_L: 0.160
CIDEr: 0.236
SPICE: 0.060

i
Bleu_1: 0.155
Bleu_2: 0.089
Bleu_3: 0.052
Bleu_4: 0.033
METEOR: 0.078
ROUGE_L: 0.189
C

In [33]:
all_topics = []
for k in web_res_dict:
    d_temp = copy.deepcopy(web_res_dict[k])
    del d_temp['Bleu_4']
    del d_temp['Bleu_2']
    del d_temp['Bleu_3']
    d_temp['topic'] = k
    all_topics.append(d_temp)
#     break
# d_temp
df_tmp = pd.DataFrame(all_topics)
df_tmp.round(2)

,Bleu_1,METEOR,ROUGE_L,CIDEr,SPICE,topic
0,0.10,0.04,0.11,0.11,0.06,a
1,0.13,0.06,0.14,0.19,0.08,b
2,0.12,0.07,0.16,0.20,0.13,c
3,0.11,0.08,0.19,0.15,0.13,d
4,0.08,0.05,0.12,0.13,0.06,e
5,0.16,0.07,0.16,0.41,0.08,f
6,0.11,0.05,0.13,0.23,0.09,g
7,0.13,0.06,0.16,0.24,0.06,h
8,0.16,0.08,0.19,0.27,0.14,i
9,0.07,0.02,0.07,0.11,0.05,j


# fat hum cider

In [28]:
# gk = df_wp.groupby('topics')
# pg_dict = dict()
# for el in gk.groups.keys():
all_dict = dict()
all_dict['data'] = copy.deepcopy(df_n)
all_dict['data'] = all_dict['data'].reset_index(drop=True)
temp = all_dict['data'][['image_id', 'ofa_caption']]
temp.columns = temp.columns.str.replace('ofa_caption', 'caption')
all_dict['results'] = temp
all_dict['images'] = id2imdf(all_dict['data'])
all_dict['human_annotations'] = h_gt2anno(all_dict['data'])

t_data = dict()
t_data['info'] = jdata['info']
t_data['images'] = all_dict['images'].to_dict('records')
t_data['type'] = jdata['type']
t_data['licenses'] = jdata['licenses']
t_data['annotations'] = all_dict['human_annotations'].to_dict('records')
file_p = f"data/all_anno.json"
with open(file_p, "w") as final:
    json.dump(t_data, final)
    
res = all_dict['results'].to_dict('records')
file_p = f"data/all_results.json"
with open(file_p, "w") as final:
    json.dump(res, final)

    
annotation_file = 'data/all_anno.json'
results_file = 'data/all_results.json'

# create coco object and coco_result object
coco = COCO(annotation_file)
coco_result = coco.loadRes(results_file)

# create coco_eval object by taking coco and coco_result
coco_eval = COCOEvalCap(coco, coco_result)

# evaluate on a subset of images by setting
# coco_eval.params['image_id'] = coco_result.getImgIds()
# please remove this line when evaluating the full validation set
coco_eval.params['image_id'] = coco_result.getImgIds()

# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
coco_eval.evaluate()

# # print output evaluation scores
# for metric, score in coco_eval.eval.items():
#     print(f'{metric}: {score:.3f}')

/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying t

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 5341, 'reflen': 3219, 'guess': [5341, 4784, 4227, 3670], 'correct': [1321, 329, 84, 27]}
ratio: 1.6592109350724886
Bleu_1: 0.247
Bleu_2: 0.130
Bleu_3: 0.070
Bleu_4: 0.040
computing METEOR score...
METEOR: 0.162
computing Rouge score...
ROUGE_L: 0.281
computing CIDEr score...
CIDEr: 0.501
computing SPICE score...
SPICE: 0.180


In [29]:
human_res_dict = dict()
for el in gk.groups.keys():
    human_res_dict[el] = calc_metrics(topic=el, anno='s')



loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 314, 'reflen': 276, 'guess': [314, 279, 244, 209], 'correct': [96, 24, 3, 0]}
ratio: 1.1376811594161678
Bleu_1: 0.306
Bleu_2: 0.162
Bleu_3: 0.069
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.134
computing Rouge score...
ROUGE_L: 0.303
computing CIDEr score...
CIDEr: 0.563
computing SPICE score...
SPICE: 0.147
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 853, 'reflen': 512, 'guess': [853, 763, 673, 583], 'correct': [212, 42, 10, 4]}
ratio: 1.6660156249967462
Bleu_1: 0.249
Bleu_2: 0.117
Bleu_3: 0.059
Bleu_4: 0.034
computing METEOR score...
METEOR: 0.159
comput

SPICE: 0.149


In [30]:
h_res = []
for el in gk.groups.keys():
    h_res.append(human_res_dict[el])
df_res = pd.DataFrame(h_res)
df_res.describe().round(2)

,Bleu_1,Bleu_2,Bleu_3,Bleu_4,METEOR,ROUGE_L,CIDEr,SPICE
count,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.00
mean,0.24,0.12,0.06,0.03,0.15,0.27,0.46,0.18
std,0.08,0.06,0.05,0.04,0.04,0.07,0.20,0.05
min,0.04,0.00,0.00,0.00,0.06,0.08,0.00,0.11
25%,0.20,0.09,0.03,0.00,0.13,0.25,0.37,0.15
50%,0.25,0.11,0.06,0.03,0.15,0.28,0.43,0.17
75%,0.27,0.14,0.07,0.04,0.17,0.30,0.57,0.20
max,0.42,0.29,0.20,0.13,0.25,0.41,0.88,0.28


In [31]:
for el in gk.groups.keys():
    print(el)
#     print(split_res_dict[el])
    for metric, score in human_res_dict[el].items():
        print(f'{metric}: {score:.3f}')
    print()

a
Bleu_1: 0.306
Bleu_2: 0.162
Bleu_3: 0.069
Bleu_4: 0.000
METEOR: 0.134
ROUGE_L: 0.303
CIDEr: 0.563
SPICE: 0.147

b
Bleu_1: 0.249
Bleu_2: 0.117
Bleu_3: 0.059
Bleu_4: 0.034
METEOR: 0.159
ROUGE_L: 0.293
CIDEr: 0.427
SPICE: 0.183

c
Bleu_1: 0.196
Bleu_2: 0.093
Bleu_3: 0.034
Bleu_4: 0.000
METEOR: 0.163
ROUGE_L: 0.248
CIDEr: 0.417
SPICE: 0.174

d
Bleu_1: 0.156
Bleu_2: 0.061
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.172
ROUGE_L: 0.211
CIDEr: 0.265
SPICE: 0.207

e
Bleu_1: 0.254
Bleu_2: 0.127
Bleu_3: 0.066
Bleu_4: 0.038
METEOR: 0.154
ROUGE_L: 0.280
CIDEr: 0.379
SPICE: 0.153

f
Bleu_1: 0.285
Bleu_2: 0.134
Bleu_3: 0.066
Bleu_4: 0.040
METEOR: 0.120
ROUGE_L: 0.298
CIDEr: 0.454
SPICE: 0.176

g
Bleu_1: 0.169
Bleu_2: 0.084
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.136
ROUGE_L: 0.205
CIDEr: 0.353
SPICE: 0.113

h
Bleu_1: 0.214
Bleu_2: 0.104
Bleu_3: 0.059
Bleu_4: 0.042
METEOR: 0.142
ROUGE_L: 0.246
CIDEr: 0.407
SPICE: 0.122

i
Bleu_1: 0.277
Bleu_2: 0.156
Bleu_3: 0.085
Bleu_4: 0.040
METEOR: 0.163
ROUGE_L: 0.307
C

In [32]:
all_topics = []
for k in human_res_dict:
    d_temp = copy.deepcopy(human_res_dict[k])
    del d_temp['Bleu_4']
    del d_temp['Bleu_2']
    del d_temp['Bleu_3']
    d_temp['topic'] = k
    all_topics.append(d_temp)
#     break
# d_temp
df_tmp = pd.DataFrame(all_topics)
df_tmp.round(2)

,Bleu_1,METEOR,ROUGE_L,CIDEr,SPICE,topic
0,0.31,0.13,0.30,0.56,0.15,a
1,0.25,0.16,0.29,0.43,0.18,b
2,0.20,0.16,0.25,0.42,0.17,c
3,0.16,0.17,0.21,0.27,0.21,d
4,0.25,0.15,0.28,0.38,0.15,e
5,0.28,0.12,0.30,0.45,0.18,f
6,0.17,0.14,0.20,0.35,0.11,g
7,0.21,0.14,0.25,0.41,0.12,h
8,0.28,0.16,0.31,0.58,0.17,i
9,0.04,0.06,0.08,0.00,0.13,j


# Fat ped Cider WEB

In [26]:
# gk = df_wp.groupby('topics')
# pg_dict = dict()
# for el in gk.groups.keys():
all_dict = dict()
all_dict['data'] = copy.deepcopy(df_n)
all_dict['data'] = all_dict['data'].reset_index(drop=True)
temp = all_dict['data'][['image_id', 'ofa_caption']]
temp.columns = temp.columns.str.replace('ofa_caption', 'caption')
all_dict['results'] = temp
all_dict['images'] = id2imdf(all_dict['data'])
all_dict['human_annotations'] = web_2anno(all_dict['data'])

t_data = dict()
t_data['info'] = jdata['info']
t_data['images'] = all_dict['images'].to_dict('records')
t_data['type'] = jdata['type']
t_data['licenses'] = jdata['licenses']
t_data['annotations'] = all_dict['human_annotations'].to_dict('records')
file_p = f"data/all_anno.json"
with open(file_p, "w") as final:
    json.dump(t_data, final)
    
res = all_dict['results'].to_dict('records')
file_p = f"data/all_results.json"
with open(file_p, "w") as final:
    json.dump(res, final)

    
annotation_file = 'data/all_anno.json'
results_file = 'data/all_results.json'

# create coco object and coco_result object
coco = COCO(annotation_file)
coco_result = coco.loadRes(results_file)

# create coco_eval object by taking coco and coco_result
coco_eval = COCOEvalCap(coco, coco_result)

# evaluate on a subset of images by setting
# coco_eval.params['image_id'] = coco_result.getImgIds()
# please remove this line when evaluating the full validation set
coco_eval.params['image_id'] = coco_result.getImgIds()

# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
coco_eval.evaluate()

# # print output evaluation scores
# for metric, score in coco_eval.eval.items():
#     print(f'{metric}: {score:.3f}')

/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying t

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 5184, 'reflen': 7910, 'guess': [5184, 4627, 4070, 3513], 'correct': [1371, 304, 93, 37]}
ratio: 0.6553729456383495
Bleu_1: 0.156
Bleu_2: 0.078
Bleu_3: 0.043
Bleu_4: 0.027
computing METEOR score...
METEOR: 0.077
computing Rouge score...
ROUGE_L: 0.175
computing CIDEr score...
CIDEr: 0.305
computing SPICE score...
SPICE: 0.109


In [27]:
web_res_dict = dict()
for el in gk.groups.keys():
#     split_res_dict[el] = calc_metrics(topic=el, anno='s')
#     con_res_dict[el] = calc_metrics(topic=el, anno='c')
    web_res_dict[el] = calc_metrics(topic=el, anno='o')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 306, 'reflen': 466, 'guess': [306, 271, 236, 201], 'correct': [65, 15, 4, 1]}
ratio: 0.6566523605136123
Bleu_1: 0.126
Bleu_2: 0.064
Bleu_3: 0.035
Bleu_4: 0.019
computing METEOR score...
METEOR: 0.056
computing Rouge score...
ROUGE_L: 0.135
computing CIDEr score...
CIDEr: 0.223
computing SPICE score...
SPICE: 0.071
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 770, 'reflen': 1242, 'guess': [770, 680, 590, 500], 'correct': [211, 40, 7, 3]}
ratio: 0.6199677938803382
Bleu_1: 0.148
Bleu_2: 0.069
Bleu_3: 0.031
Bleu_4: 0.018
computing METEOR score...
METEOR: 0.080
comput

SPICE: 0.065


In [28]:
all_res = []
for el in gk.groups.keys():
    all_res.append(web_res_dict[el])
df_res = pd.DataFrame(all_res)
df_res.describe().round(2)

,Bleu_1,Bleu_2,Bleu_3,Bleu_4,METEOR,ROUGE_L,CIDEr,SPICE
count,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.00
mean,0.14,0.07,0.04,0.02,0.07,0.17,0.28,0.10
std,0.03,0.03,0.02,0.02,0.02,0.04,0.10,0.04
min,0.07,0.00,0.00,0.00,0.04,0.10,0.12,0.04
25%,0.13,0.05,0.02,0.00,0.06,0.14,0.20,0.06
50%,0.14,0.06,0.03,0.02,0.07,0.18,0.22,0.09
75%,0.16,0.09,0.05,0.03,0.08,0.19,0.36,0.14
max,0.19,0.11,0.06,0.05,0.10,0.22,0.43,0.17


In [29]:
for el in gk.groups.keys():
    print(el)
#     print(split_res_dict[el])
    for metric, score in web_res_dict[el].items():
        print(f'{metric}: {score:.3f}')
    print()

a
Bleu_1: 0.126
Bleu_2: 0.064
Bleu_3: 0.035
Bleu_4: 0.019
METEOR: 0.056
ROUGE_L: 0.135
CIDEr: 0.223
SPICE: 0.071

b
Bleu_1: 0.148
Bleu_2: 0.069
Bleu_3: 0.031
Bleu_4: 0.018
METEOR: 0.080
ROUGE_L: 0.187
CIDEr: 0.337
SPICE: 0.114

c
Bleu_1: 0.185
Bleu_2: 0.102
Bleu_3: 0.063
Bleu_4: 0.044
METEOR: 0.095
ROUGE_L: 0.198
CIDEr: 0.356
SPICE: 0.167

d
Bleu_1: 0.140
Bleu_2: 0.072
Bleu_3: 0.037
Bleu_4: 0.019
METEOR: 0.094
ROUGE_L: 0.216
CIDEr: 0.191
SPICE: 0.156

e
Bleu_1: 0.136
Bleu_2: 0.050
Bleu_3: 0.024
Bleu_4: 0.000
METEOR: 0.075
ROUGE_L: 0.196
CIDEr: 0.183
SPICE: 0.093

f
Bleu_1: 0.135
Bleu_2: 0.088
Bleu_3: 0.063
Bleu_4: 0.048
METEOR: 0.049
ROUGE_L: 0.124
CIDEr: 0.424
SPICE: 0.054

g
Bleu_1: 0.138
Bleu_2: 0.052
Bleu_3: 0.024
Bleu_4: 0.000
METEOR: 0.055
ROUGE_L: 0.146
CIDEr: 0.217
SPICE: 0.081

h
Bleu_1: 0.167
Bleu_2: 0.086
Bleu_3: 0.053
Bleu_4: 0.032
METEOR: 0.079
ROUGE_L: 0.182
CIDEr: 0.219
SPICE: 0.095

i
Bleu_1: 0.162
Bleu_2: 0.084
Bleu_3: 0.050
Bleu_4: 0.034
METEOR: 0.078
ROUGE_L: 0.191
C

In [30]:
all_topics = []
for k in web_res_dict:
    d_temp = copy.deepcopy(web_res_dict[k])
    del d_temp['Bleu_4']
    del d_temp['Bleu_2']
    del d_temp['Bleu_3']
    d_temp['topic'] = k
    all_topics.append(d_temp)
#     break
# d_temp
df_tmp = pd.DataFrame(all_topics)
df_tmp.round(2)

,Bleu_1,METEOR,ROUGE_L,CIDEr,SPICE,topic
0,0.13,0.06,0.13,0.22,0.07,a
1,0.15,0.08,0.19,0.34,0.11,b
2,0.18,0.09,0.20,0.36,0.17,c
3,0.14,0.09,0.22,0.19,0.16,d
4,0.14,0.07,0.20,0.18,0.09,e
5,0.13,0.05,0.12,0.42,0.05,f
6,0.14,0.06,0.15,0.22,0.08,g
7,0.17,0.08,0.18,0.22,0.09,h
8,0.16,0.08,0.19,0.35,0.13,i
9,0.18,0.07,0.18,0.12,0.06,j


#  hum sim

In [26]:
# gk = df_wp.groupby('topics')
# pg_dict = dict()
# for el in gk.groups.keys():
all_dict = dict()
all_dict['data'] = copy.deepcopy(df_n)
all_dict['data'] = all_dict['data'].reset_index(drop=True)
temp = all_dict['data'][['image_id', 'ofa_caption']]
temp.columns = temp.columns.str.replace('ofa_caption', 'caption')
all_dict['results'] = temp
all_dict['images'] = id2imdf(all_dict['data'])
all_dict['human_annotations'] = h_gt2anno(all_dict['data'])

t_data = dict()
t_data['info'] = jdata['info']
t_data['images'] = all_dict['images'].to_dict('records')
t_data['type'] = jdata['type']
t_data['licenses'] = jdata['licenses']
t_data['annotations'] = all_dict['human_annotations'].to_dict('records')
file_p = f"data/all_anno.json"
with open(file_p, "w") as final:
    json.dump(t_data, final)
    
res = all_dict['results'].to_dict('records')
file_p = f"data/all_results.json"
with open(file_p, "w") as final:
    json.dump(res, final)

    
annotation_file = 'data/all_anno.json'
results_file = 'data/all_results.json'

# create coco object and coco_result object
coco = COCO(annotation_file)
coco_result = coco.loadRes(results_file)

# create coco_eval object by taking coco and coco_result
coco_eval = COCOEvalCap(coco, coco_result)

# evaluate on a subset of images by setting
# coco_eval.params['image_id'] = coco_result.getImgIds()
# please remove this line when evaluating the full validation set
coco_eval.params['image_id'] = coco_result.getImgIds()

# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
coco_eval.evaluate()

# # print output evaluation scores
# for metric, score in coco_eval.eval.items():
#     print(f'{metric}: {score:.3f}')

/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying t

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 4917, 'reflen': 3219, 'guess': [4917, 4360, 3803, 3246], 'correct': [1132, 262, 68, 21]}
ratio: 1.5274930102511564
Bleu_1: 0.230
Bleu_2: 0.118
Bleu_3: 0.063
Bleu_4: 0.036
computing METEOR score...
METEOR: 0.153
computing Rouge score...
ROUGE_L: 0.247
computing CIDEr score...
CIDEr: 0.491
computing SPICE score...
SPICE: 0.184


In [27]:
human_res_dict = dict()
for el in gk.groups.keys():
    human_res_dict[el] = calc_metrics(topic=el, anno='s')



loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 289, 'reflen': 276, 'guess': [289, 254, 219, 184], 'correct': [90, 19, 1, 0]}
ratio: 1.0471014492715685
Bleu_1: 0.311
Bleu_2: 0.153
Bleu_3: 0.047
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.132
computing Rouge score...
ROUGE_L: 0.264
computing CIDEr score...
CIDEr: 0.551
computing SPICE score...
SPICE: 0.187
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 740, 'reflen': 512, 'guess': [740, 650, 560, 470], 'correct': [182, 43, 15, 5]}
ratio: 1.4453124999971771
Bleu_1: 0.246
Bleu_2: 0.128
Bleu_3: 0.076
Bleu_4: 0.046
computing METEOR score...
METEOR: 0.153
comput

SPICE: 0.154


In [28]:
h_res = []
for el in gk.groups.keys():
    h_res.append(human_res_dict[el])
df_res = pd.DataFrame(h_res)
df_res.describe().round(2)

,Bleu_1,Bleu_2,Bleu_3,Bleu_4,METEOR,ROUGE_L,CIDEr,SPICE
count,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.00
mean,0.22,0.10,0.05,0.02,0.14,0.23,0.41,0.18
std,0.08,0.06,0.05,0.03,0.04,0.07,0.19,0.05
min,0.04,0.00,0.00,0.00,0.07,0.08,0.00,0.10
25%,0.18,0.07,0.00,0.00,0.12,0.21,0.30,0.16
50%,0.21,0.10,0.05,0.00,0.14,0.22,0.41,0.18
75%,0.24,0.13,0.06,0.03,0.16,0.26,0.53,0.20
max,0.40,0.25,0.16,0.10,0.24,0.38,0.77,0.30


In [29]:
for el in gk.groups.keys():
    print(el)
#     print(split_res_dict[el])
    for metric, score in human_res_dict[el].items():
        print(f'{metric}: {score:.3f}')
    print()

a
Bleu_1: 0.311
Bleu_2: 0.153
Bleu_3: 0.047
Bleu_4: 0.000
METEOR: 0.132
ROUGE_L: 0.264
CIDEr: 0.551
SPICE: 0.187

b
Bleu_1: 0.246
Bleu_2: 0.128
Bleu_3: 0.076
Bleu_4: 0.046
METEOR: 0.153
ROUGE_L: 0.262
CIDEr: 0.544
SPICE: 0.196

c
Bleu_1: 0.189
Bleu_2: 0.075
Bleu_3: 0.036
Bleu_4: 0.000
METEOR: 0.158
ROUGE_L: 0.202
CIDEr: 0.331
SPICE: 0.162

d
Bleu_1: 0.162
Bleu_2: 0.064
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.167
ROUGE_L: 0.214
CIDEr: 0.277
SPICE: 0.213

e
Bleu_1: 0.214
Bleu_2: 0.116
Bleu_3: 0.062
Bleu_4: 0.032
METEOR: 0.140
ROUGE_L: 0.217
CIDEr: 0.406
SPICE: 0.176

f
Bleu_1: 0.242
Bleu_2: 0.114
Bleu_3: 0.058
Bleu_4: 0.000
METEOR: 0.119
ROUGE_L: 0.278
CIDEr: 0.507
SPICE: 0.187

g
Bleu_1: 0.145
Bleu_2: 0.033
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.123
ROUGE_L: 0.165
CIDEr: 0.253
SPICE: 0.102

h
Bleu_1: 0.183
Bleu_2: 0.081
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.138
ROUGE_L: 0.224
CIDEr: 0.316
SPICE: 0.103

i
Bleu_1: 0.239
Bleu_2: 0.133
Bleu_3: 0.065
Bleu_4: 0.000
METEOR: 0.151
ROUGE_L: 0.264
C

In [30]:
all_topics = []
for k in human_res_dict:
    d_temp = copy.deepcopy(human_res_dict[k])
    del d_temp['Bleu_4']
    del d_temp['Bleu_2']
    del d_temp['Bleu_3']
    d_temp['topic'] = k
    all_topics.append(d_temp)
#     break
# d_temp
df_tmp = pd.DataFrame(all_topics)
df_tmp.round(2)

,Bleu_1,METEOR,ROUGE_L,CIDEr,SPICE,topic
0,0.31,0.13,0.26,0.55,0.19,a
1,0.25,0.15,0.26,0.54,0.20,b
2,0.19,0.16,0.20,0.33,0.16,c
3,0.16,0.17,0.21,0.28,0.21,d
4,0.21,0.14,0.22,0.41,0.18,e
5,0.24,0.12,0.28,0.51,0.19,f
6,0.14,0.12,0.17,0.25,0.10,g
7,0.18,0.14,0.22,0.32,0.10,h
8,0.24,0.15,0.26,0.45,0.15,i
9,0.04,0.07,0.08,0.00,0.17,j


#  web sim

In [31]:
# gk = df_wp.groupby('topics')
# pg_dict = dict()
# for el in gk.groups.keys():
all_dict = dict()
all_dict['data'] = copy.deepcopy(df_n)
all_dict['data'] = all_dict['data'].reset_index(drop=True)
temp = all_dict['data'][['image_id', 'ofa_caption']]
temp.columns = temp.columns.str.replace('ofa_caption', 'caption')
all_dict['results'] = temp
all_dict['images'] = id2imdf(all_dict['data'])
all_dict['human_annotations'] = web_2anno(all_dict['data'])

t_data = dict()
t_data['info'] = jdata['info']
t_data['images'] = all_dict['images'].to_dict('records')
t_data['type'] = jdata['type']
t_data['licenses'] = jdata['licenses']
t_data['annotations'] = all_dict['human_annotations'].to_dict('records')
file_p = f"data/all_anno.json"
with open(file_p, "w") as final:
    json.dump(t_data, final)
    
res = all_dict['results'].to_dict('records')
file_p = f"data/all_results.json"
with open(file_p, "w") as final:
    json.dump(res, final)

    
annotation_file = 'data/all_anno.json'
results_file = 'data/all_results.json'

# create coco object and coco_result object
coco = COCO(annotation_file)
coco_result = coco.loadRes(results_file)

# create coco_eval object by taking coco and coco_result
coco_eval = COCOEvalCap(coco, coco_result)

# evaluate on a subset of images by setting
# coco_eval.params['image_id'] = coco_result.getImgIds()
# please remove this line when evaluating the full validation set
coco_eval.params['image_id'] = coco_result.getImgIds()

# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
coco_eval.evaluate()

# # print output evaluation scores
# for metric, score in coco_eval.eval.items():
#     print(f'{metric}: {score:.3f}')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...


/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying t

setting up scorers...
computing Bleu score...
{'testlen': 4917, 'reflen': 7910, 'guess': [4917, 4360, 3803, 3246], 'correct': [1347, 346, 119, 51]}
ratio: 0.6216182048039669
Bleu_1: 0.149
Bleu_2: 0.080
Bleu_3: 0.048
Bleu_4: 0.031
computing METEOR score...
METEOR: 0.078
computing Rouge score...
ROUGE_L: 0.178
computing CIDEr score...
CIDEr: 0.385
computing SPICE score...
SPICE: 0.119


In [32]:
human_res_dict = dict()
for el in gk.groups.keys():
    human_res_dict[el] = calc_metrics(topic=el, anno='o')



loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 289, 'reflen': 466, 'guess': [289, 254, 219, 184], 'correct': [68, 16, 4, 2]}
ratio: 0.6201716738184117
Bleu_1: 0.128
Bleu_2: 0.066
Bleu_3: 0.035
Bleu_4: 0.022
computing METEOR score...
METEOR: 0.060
computing Rouge score...
ROUGE_L: 0.140
computing CIDEr score...
CIDEr: 0.181
computing SPICE score...
SPICE: 0.111
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 740, 'reflen': 1242, 'guess': [740, 650, 560, 470], 'correct': [207, 42, 6, 3]}
ratio: 0.5958132045083769
Bleu_1: 0.142
Bleu_2: 0.068
Bleu_3: 0.029
Bleu_4: 0.017
computing METEOR score...
METEOR: 0.080
comput

SPICE: 0.081


In [33]:
h_res = []
for el in gk.groups.keys():
    h_res.append(human_res_dict[el])
df_res = pd.DataFrame(h_res)
df_res.describe().round(2)

,Bleu_1,Bleu_2,Bleu_3,Bleu_4,METEOR,ROUGE_L,CIDEr,SPICE
count,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.00
mean,0.13,0.07,0.04,0.02,0.07,0.16,0.32,0.10
std,0.04,0.03,0.02,0.02,0.02,0.05,0.22,0.05
min,0.07,0.00,0.00,0.00,0.03,0.08,0.13,0.04
25%,0.12,0.05,0.02,0.01,0.06,0.13,0.19,0.07
50%,0.14,0.07,0.04,0.02,0.07,0.15,0.24,0.09
75%,0.16,0.09,0.05,0.03,0.09,0.20,0.37,0.12
max,0.18,0.11,0.08,0.07,0.10,0.25,1.04,0.21


In [34]:
for el in gk.groups.keys():
    print(el)
#     print(split_res_dict[el])
    for metric, score in human_res_dict[el].items():
        print(f'{metric}: {score:.3f}')
    print()

a
Bleu_1: 0.128
Bleu_2: 0.066
Bleu_3: 0.035
Bleu_4: 0.022
METEOR: 0.060
ROUGE_L: 0.140
CIDEr: 0.181
SPICE: 0.111

b
Bleu_1: 0.142
Bleu_2: 0.068
Bleu_3: 0.029
Bleu_4: 0.017
METEOR: 0.080
ROUGE_L: 0.178
CIDEr: 0.330
SPICE: 0.112

c
Bleu_1: 0.165
Bleu_2: 0.100
Bleu_3: 0.065
Bleu_4: 0.043
METEOR: 0.094
ROUGE_L: 0.204
CIDEr: 0.360
SPICE: 0.155

d
Bleu_1: 0.121
Bleu_2: 0.065
Bleu_3: 0.034
Bleu_4: 0.017
METEOR: 0.089
ROUGE_L: 0.209
CIDEr: 0.198
SPICE: 0.138

e
Bleu_1: 0.097
Bleu_2: 0.038
Bleu_3: 0.018
Bleu_4: 0.000
METEOR: 0.059
ROUGE_L: 0.141
CIDEr: 0.135
SPICE: 0.093

f
Bleu_1: 0.160
Bleu_2: 0.105
Bleu_3: 0.066
Bleu_4: 0.038
METEOR: 0.065
ROUGE_L: 0.141
CIDEr: 0.427
SPICE: 0.057

g
Bleu_1: 0.144
Bleu_2: 0.059
Bleu_3: 0.027
Bleu_4: 0.000
METEOR: 0.057
ROUGE_L: 0.149
CIDEr: 0.224
SPICE: 0.081

h
Bleu_1: 0.164
Bleu_2: 0.080
Bleu_3: 0.048
Bleu_4: 0.029
METEOR: 0.085
ROUGE_L: 0.197
CIDEr: 0.196
SPICE: 0.087

i
Bleu_1: 0.174
Bleu_2: 0.114
Bleu_3: 0.085
Bleu_4: 0.066
METEOR: 0.099
ROUGE_L: 0.246
C

In [35]:
all_topics = []
for k in human_res_dict:
    d_temp = copy.deepcopy(human_res_dict[k])
    del d_temp['Bleu_4']
    del d_temp['Bleu_2']
    del d_temp['Bleu_3']
    d_temp['topic'] = k
    all_topics.append(d_temp)
#     break
# d_temp
df_tmp = pd.DataFrame(all_topics)
df_tmp.round(2)

,Bleu_1,METEOR,ROUGE_L,CIDEr,SPICE,topic
0,0.13,0.06,0.14,0.18,0.11,a
1,0.14,0.08,0.18,0.33,0.11,b
2,0.16,0.09,0.20,0.36,0.16,c
3,0.12,0.09,0.21,0.20,0.14,d
4,0.10,0.06,0.14,0.13,0.09,e
5,0.16,0.07,0.14,0.43,0.06,f
6,0.14,0.06,0.15,0.22,0.08,g
7,0.16,0.08,0.20,0.20,0.09,h
8,0.17,0.10,0.25,1.04,0.21,i
9,0.07,0.03,0.08,0.15,0.06,j
